In [1]:
!pip install --upgrade git+https://github.com/keras-team/keras-cv -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 5.7 MB/s eta 0:00:00


In [2]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras
from itertools import chain
# import matplotlib.pyplot as plt
import cv2


import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

Using TensorFlow backend


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model = tf.keras.models.load_model("/content/drive/MyDrive/ChickMed/ML/Model_YOLOv8/yolo_v8_xs_backbone_coco.h5",
    custom_objects={
    "YOLOV8Detector": keras_cv.models.YOLOV8Detector,
    "YOLOV8Backbone": keras_cv.models.YOLOV8Backbone
            },
    compile = False)

In [6]:
class_ids = [
    "salmo",
    'cocci',
    'healthy',
    'ncd',
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))
class_mapping

{0: 'salmo', 1: 'cocci', 2: 'healthy', 3: 'ncd'}

1/1 [==============================] - 0s 89ms/step


In [7]:
image = tf.io.read_file("/content/drive/MyDrive/ChickMed/ML/20160320_135433.jpg")
image = tf.image.decode_jpeg(image, channels=3)
image = tf.image.resize(image,(256,256))
image = tf.expand_dims(image,axis = 0)
predictions = model.predict(image)
y_pred = bounding_box.to_ragged(predictions)
visualization.plot_bounding_box_gallery(
        image,
        value_range=(0, 255),
        bounding_box_format="xywh",
        y_pred=y_pred,
        scale=2,
        rows=1,
        cols=1,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )

In [8]:
def get_prediction(image_path,model):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image,(256,256))
    image = tf.expand_dims(image,axis = 0)
    predictions = model.predict(image)
    return predictions

In [9]:
def draw_prediction(image_path,model):
    #compute class mapping
    class_ids = ["salmo",'cocci','healthy','ncd',]
    class_mapping = dict(zip(range(len(class_ids)), class_ids))
    color_list = [(231,76,60),(52,152,219),(39, 231, 96),(243,156,18)]
    #get original image
    original_image = cv2.imread(image_path)
    #original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    width = original_image.shape[1]
    height = original_image.shape[0]

    #get scaled image
    scale_width = width / 256
    scale_height = height / 256

    #get predictions
    predictions = get_prediction(image_path,model)

    #get boxes, confindences and classes
    num_detections = int(predictions['num_detections'][0])
    classes = predictions['classes'][0]
    confindences = predictions['confidence'][0]
    boxes = predictions['boxes'][0]
    confidences_tmp = []
    classes_tmp = []

    for i in range(num_detections):
        class_id = int(classes[i])
        class_name = class_mapping[class_id]
        classes_tmp.append(class_name)
        confidences_tmp.append(confindences[i])
        x,y,w,h = boxes[i]
        x1 = int(x * scale_width)
        y1 = int(y * scale_height)
        w = int(w * scale_width)
        h = int(h * scale_height)
        x2 = x1 + w
        y2 = y1 + h
        #check if box is out of bounds
        if x1 < 0:
            x1 = int(0 + width*0.05)
        if y1 < 0:
            y1 = int(0 + height*0.05)
        if x2 > width:
            x2 = int(width - width*0.05)
        if y2 > height:
            y2 = int(height - height*0.05)

        cv2.rectangle(original_image, (x1, y1), (x2, y2), color_list[class_id], 1)

        linewidth = min(int((x2-x1)*0.2),int((y2-y1)*0.2))
        cv2.line(original_image, (x1, y1), (x1+linewidth, y1), color_list[class_id], 4)
        cv2.line(original_image, (x1, y1), (x1, y1+linewidth), color_list[class_id], 4)
        cv2.line(original_image, (x2, y1), (x2-linewidth, y1), color_list[class_id], 4)
        cv2.line(original_image, (x2, y1), (x2, y1+linewidth), color_list[class_id], 4)

        cv2.line(original_image, (x1, y2), (x1+linewidth, y2), color_list[class_id], 4)
        cv2.line(original_image, (x1, y2), (x1, y2-linewidth), color_list[class_id], 4)
        cv2.line(original_image, (x2, y2), (x2-linewidth, y2), color_list[class_id], 4)
        cv2.line(original_image, (x2, y2), (x2, y2-linewidth), color_list[class_id], 4)

    prediction_final = {
        "num_detection":num_detections,
        "classes":classes_tmp,
        "confindences":confidences_tmp,
    }



    #save image
    cv2.imwrite('prediction.jpg',original_image)
    return prediction_final

In [10]:
path = "/content/drive/MyDrive/ChickMed/ML/20160320_135433.jpg"
predictions_final = draw_prediction(path,model)

1/1 [==============================] - 0s 145ms/step
